In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "../drug200.csv"
data = pd.read_csv(file_path)

data.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


# Funcion de entropia
Se usa para medir la incertidumbre o impureza en un conjunto de datos, permitiendo determinar que tan mezcladas estan las clases dentro de un nodo

Entropy(S) = - ∑ pᵢ * log₂(pᵢ) ; i = 1 to n

- pi: proporcion de elementos de la clase i
- n: numero total de clases

En el algoritmo se toma una columna de datos que representa a las clases objetivo y calcula la entriopia de esa columa, asi hasta obtener la de todas

In [3]:
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

# Obtencion de la informacion
A partir de la entropia se puede calcular la ganancia de informacion, esto mide la reduccion en la entropia al dividir el conjunto de datos usando una caracteristica especifica

IG(S, A) = Entropy(S) - ∑((|Sᵥ| / |S|) * Entropy(Sᵥ))

Primero se calcula la entropia de todos los datos, despues se dividen en sus atributos y se calcula la entopia de cada subconjunto para finalmente multiplicarlo por la proporcion de los datos que representa.

En este caso como la categoria que se quiere saber es la de medicinas (Que en el dataset esta como Drug) es la que se coloca como atributo del target_name

In [4]:
def InfoGain(data, split_attribute_name, target_name="Drug"):
    total_entropy = entropy(data[target_name])
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)
    weighted_entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    information_gain = total_entropy - weighted_entropy
    return information_gain

# Construccion del arbol
A partir de la ganancia de informacion se puede constuir el arbol, en este caso se usa el algoritmo ID3, que consiste en ir dividiendo las caraceteristicas en dos o mas grupos en cada paso de forma iterativa.

Este se suele usar cuando la mayoria o todos los datos que se cuentan no son cuantitativos.

Lo que hace es seleccionar la mejor caracteristica para dividir el subconjunto de datos en cada paso, haciendolo hasta que o ya esten clasificados o no haya mas caracteristicas para dividir

## Caso base
- Si el conjunto esta vacio se devuelve el nodo padre
- Si todos los elementos tienen la misma clase, entonces se devuelve esa clase
- Si no hay mas caracteristicas para dividir, entonces se devuelve la clase del nodo padre

## Recursividad
Se calcula la ganancia de informacion para cada caracteristica restante y se selecciona la caracteristica que tiene mayor ganancia, se crea un modelo en el arbol para esa caracteristica y se remueve ese caracteristica de la lista.

Para cada valor unico de la caracteristica seleccionada se divide el conjunto de datosy se llama recusrivamente a la funcion para volver a construir los subarboles

In [5]:
def ID3(data, original_data, features, target_attribute_name="Drug", parent_node_class=None):
    # Caso base: Si el conjunto de datos está vacío, devolver el nodo padre
    if len(data) == 0:
        return parent_node_class
    
    # Caso base: Si todos los elementos tienen la misma clase, devolver esa clase
    elif len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    # Caso base: Si no hay más características, devolver el modo del nodo padre
    elif len(features) == 0:
        return parent_node_class
    
    # Caso recursivo
    else:
        # Establecer el nodo padre
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]
        
        # Seleccionar la característica que tiene la mayor ganancia de información
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features] 
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        # Crear la estructura del árbol
        tree = {best_feature: {}}
        
        # Remover la mejor característica de la lista de características
        features = [i for i in features if i != best_feature]
        
        # Crecer el árbol para cada valor de la característica seleccionada
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data, data, features, target_attribute_name, parent_node_class)
            tree[best_feature][value] = subtree
        
        return tree


# Esta parte se puede ignorar solo fue para comprobar que funcionara el arbol, permitio concluir que las caracteristicas que mas definen el medicamento es e lcolesterol y el Na

In [6]:
features = data.columns[:-1]
target = "Drug"

tree = ID3(data, data, features, target)

import pprint
pprint.pprint(tree)

{'Na_to_K': {6.269: 'drugA',
             6.683: 'drugX',
             6.769: 'drugC',
             7.261: 'drugX',
             7.285: 'drugX',
             7.298: 'drugC',
             7.34: 'drugX',
             7.477: 'drugX',
             7.49: 'drugA',
             7.761: 'drugX',
             7.798: 'drugX',
             7.845: 'drugX',
             8.011: 'drugA',
             8.107: 'drugX',
             8.151: 'drugC',
             8.37: 'drugX',
             8.607: 'drugX',
             8.621: 'drugB',
             8.7: 'drugA',
             8.75: 'drugX',
             8.966: 'drugX',
             8.968: 'drugX',
             9.084: 'drugX',
             9.17: 'drugX',
             9.281: 'drugX',
             9.381: 'drugX',
             9.443: 'drugX',
             9.445: 'drugA',
             9.475: 'drugA',
             9.514: 'drugX',
             9.567: 'drugB',
             9.664: 'drugA',
             9.677: 'drugB',
             9.709: 'drugX',
             9.712: '

# Prediccion
Aunque el arbol como tal ya esta hecho, este necesita una forma de poder predecir un dato en caso de que se registre un nuevo paciente, por lo que lo que se hara es recorrer el arbol a partir de las caracteristicas que proporciona el paciente

- Primero se busca el valor correspondiente en la muestra del paciente, en caso de que ese valor se encuentre en el arbol entonces se sigue su respectivo subarbol, pero si no entonces se busca el valor mas cercano y se recorre su arbol

- Finalmente si se llega a una hoja entonces se retorna el medicamento

In [7]:
def predict(tree, sample):
    for key in tree.keys():
        value = sample[key]
        if value in tree[key]:
            subtree = tree[key][value]
            if isinstance(subtree, dict):
                return predict(subtree, sample)
            else:
                return subtree
        else:
            closest_value = min(tree[key].keys(), key=lambda x:abs(x-value))
            subtree = tree[key][closest_value]
            if isinstance(subtree, dict):
                return predict(subtree, sample)
            else:
                return subtree

# Funcion para obtener la informacion del paciente, este se explica solo

In [8]:
def get_user_input():
    age = int(input("Enter patient age: "))
    sex = input("Enter pacient sex (M/F): ")
    bp = input("Enter patient blood pressure (HIGH/NORMAL): ")
    cholesterol = input("Enter patient cholesterol (HIGH/NORMAL): ")
    sodium = float(input("Enter patient sodium level: "))
    
    sample = {
        "Age": age,
        "Sex": sex,
        "BP": bp,
        "Cholesterol": cholesterol,
        "Na_to_K": sodium
    }
    
    return sample

In [9]:
features = data.columns[:-1]
target = "Drug"

decision_tree = ID3(data, data, features, target)

sample = get_user_input()
prediction = predict(decision_tree, sample)

print("The predicted drug for the patient is: ", prediction)

The predicted drug for the patient is:  drugA
